### Import Required Libraries and Set Up Environment Variables

In [ ]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [ ]:
# Set environment variables from the .env in the local environment
load_dotenv('.env')

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [ ]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"{url}&api-key={nyt_api_key}&api-key={tmdb_api_key}&fq={filter_query}&begin_date={begin_date}&end_date={end_date}&sort={sort}&fl={field_list}"
query_url

In [ ]:
# Create an empty list to store the reviews
review_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    params = {
        'fq': filter_query,
        'sort': sort,
        'fl': field_list,
        'begin_date': begin_date,
        'end_date': end_date,
        'api-key': nyt_api_key,
        'page': page
    }
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(url, params=params)
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    if response.status_code == 200:
        reviews = response.json()
        try:
        # loop through the reviews["response"]["docs"] and append each review to the list
            for review in reviews["response"]["docs"]:
                review_list.append(review)
        # Print the page that was just retrieved
            print(f"Checked page {page}")
        except KeyError:
            print(f"Page {page} had no results.")
        # Print the page number that had no results then break from the loop
    else:
        print(f"Failed to retrieve page {page}, status code: {response.status_code}")
        break

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
first_five = review_list[0:5] # get first five elements of the list
formatted_first_five = json.dumps(first_five, indent=4)
print(formatted_first_five)

In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df_reviews = pd.json_normalize(review_list)

df_reviews.head()

In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

df_reviews['title'] =  df_reviews['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

df_reviews

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
df_reviews['keywords'] = df_reviews['keywords'].apply(extract_keywords)
df_reviews

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = df_reviews['title'].to_list()
title_list

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [ ]:
# Create an empty list to store the results
results = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter  = 1

# Loop through the titles
for title in title_list:
    # Check if we need to sleep before making a request
    if (request_counter % 50) == 0:
        print(f'Sleeping at {request_counter} requests')
        time.sleep(5)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    movies = requests.get(url+title+tmdb_key_string)
    movie_data = movies.json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = movie_data['results'][0]['id']

        # Make a request for a the full movie details
        id_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"

        # Execute "GET" request with url
        full_movie_details = requests.get(id_url).json()
        
        # Extract the genre names into a list
        genres = []
        for genre in full_movie_details.get('genres'):
            genres.append(genre.get('name'))

        # Extract the spoken_languages' English name into a list
        spoken_languages = []
        for language in full_movie_details.get('spoken_languages'):
            spoken_languages.append(language.get('english_name'))

        # Extract the production_countries' name into a list
        prod_countries = []
        for country in full_movie_details.get('production_countries'):
            prod_countries.append(country.get('name'))

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        results.append({
            'title': full_movie_details['title'],
            'original_title': full_movie_details.get('original_title'),
            'budget': full_movie_details.get('budget'),
            'genre': genres,
            'language': full_movie_details.get('original_language'),
            'spoken_languages': spoken_languages,
            'homepage': full_movie_details.get('homepage'),
            'overview': full_movie_details.get('overview'),
            'popularity': full_movie_details.get('popularity'),
            'runtime': full_movie_details.get('runtime'),
            'revenue': full_movie_details.get('revenue'),
            'release_date': full_movie_details.get('release_date'),
            'vote_average': full_movie_details.get('vote_average'),
            'vote_count': full_movie_details.get('vote_count'),
            'production_countries': prod_countries
        })
        
        # Print out the title that was found
        print(f'found {title}')
    except Exception as e: 
        print(f'{title} not found')


In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(results[:5], indent=4))

In [ ]:
# Convert the results to a DataFrame
tmdb_results_df = pd.DataFrame(results)
tmdb_results_df

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(tmdb_results_df, df_reviews, on='title')
merged_df.head()

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genre', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix

for column in columns_to_fix:

    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype("str")

    # Loop through characters to remove
    for character in characters_to_remove:
        merged_df[column] = merged_df[column].str.replace(character, "", regex=False)

# Display the fixed DataFrame
merged_df.head(5)

In [ ]:
# Drop "byline.person" column
merged_df.drop(columns='byline.person', inplace=True)

In [ ]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates().reset_index(inplace=True)
merged_df.head(5)

In [ ]:
# Export data to CSV without the index
merged_df.to_csv("final_csv.csv", index=False)